# PennyLane Tailgating Demo
A numerical example, demonstrating the tailgating procedure in the case of computing derivatives of a molecular energy function with respect to some set of nuclear coordinates.

In [45]:
%load_ext autoreload
%autoreload 2
import tailgating as tg
import pennylane as qml
import pennylane.numpy as np
from pennylane import qchem
import scipy
from tqdm.notebook import tqdm
import autohf as hf
import copy
import itertools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 1. Performing ADAPT-VQE

In this Notebook, we will attempt to compute the normal mode frequencies of a BeH$_2$ molecule. We begin by declaring some basic information about the molecule:

In [2]:
# Basic information
symbols = ["Be", "H", "H"]
geometry = tg.angs_bohr * np.array([
    [0.0000000000, 0.000000000, 0.0000000000], 
    [0.0000000000, -1.31642901431, 0.0000000000], 
    [0.0000000000, 1.3164290143, 0.0000000000]
])

# Specifies the electrons and orbitals
electrons, active_electrons = 6, 4
orbitals, active_orbitals = 7, 6
wires = list(range(2 * active_orbitals))

# Defines the molecule and the active space
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=6)

We can put all of this information inside of a class, representing the molecule:

In [3]:
class BeH2:

    basis_name = "sto-3g"
    symbols = symbols
    active_electrons = active_electrons
    active_orbitals = active_orbitals
    charge = 0
    multiplicity = 1
    
    geometry = geometry

mol = BeH2()

We will compare calculations done with ADAPT-VQE, with and without the tailgating procedure, and demonstrate that the non-tailgated circuit yields inaccurate derivatives while those yielded from tailgating are highly accurate. The first step is to perform ADAPT-VQE, using a gate pool of single and double excitations to get our initial circuit, for preparing ground states:

In [4]:
# Generates the gate pool to use for ADAPT-VQE
gate_pool = tg.batch_gate_pool(mol)

# Generates the sparse molecular Hamiltonian (used in the VQE simulations)
H = hf.sparse_H(mol, wires)(mol.geometry)

# Other information: the device, Hartree-Fock state, optimizer, and numbers of optimization steps
dev = qml.device('default.qubit', wires=wires)
hf_state = qchem.hf_state(active_electrons, 2 * active_orbitals)
optimizer = qml.GradientDescentOptimizer(stepsize=0.3)
vqe_steps = 100

# Performs a batched ADAPT-VQE procedure, applying all double excitations in the gate pool, then all single excitations
original_seq, original_params = tg.batch_adapt_vqe(H, dev, gate_pool, hf_state, optimizer, vqe_steps, bar=True, sparse=True)

  0%|          | 0/100 [00:00<?, ?it/s]

We can then build the circuit that we use for VQE.

In [5]:
# Circuit yielded from ADAPT-VQE
def circuit(params):
    qml.BasisState(hf_state, wires=wires)
    for p, gate in zip(params, original_seq):
        gate(p)

# Circuit that returns state vector
@qml.qnode(dev)
def state_circuit(params):
    circuit(params)
    return qml.state()

Then, we can perform VQE to attempt to find the ground state of the molecule:

In [6]:
energy, params = tg.vqe(circuit, H, dev, optimizer, 100, original_params, sparse=True)

  0%|          | 0/100 [00:00<?, ?it/s]

Since we are working with a simulator, we have direct access to the state vector, and can determine how close our approxiamte ground state (yielded from VQE) is to the true ground state of the Hamiltonian (we compute the exact ground state through exact diagonalization):

In [7]:
# State vector yielded from ADAPT-VQE circuit
state = tg.compute_state(circuit, dev, params)

# Returns the dot product between the true ground state of H and the prepared ground state
print(abs(np.dot(np.linalg.eigh(H.matrix())[1].T[0], state)))

0.9998450120873879


As can be seen, the magnitude of the dot product is very close to one, implying that the prepared state is very close to the true ground state. However, as we will soon demonstrate, this does not necessarily mean that our circuit will yield accurate energy derivatives.

### Step 2. Computing the energy derivative with the non-tailgated circuit

The next step is to calculate the second-order energy derivative using the non-tailgated circuit. Recall that the formula for such a derivative is given by:

$$
\frac{\partial^2 E_k(\textbf{R})}{\partial R_i \partial R_j} = \displaystyle\sum_{a} \Big[ \frac{\partial \boldsymbol\theta^{k}_a(\textbf{R})}{\partial R_i} \frac{\partial}{\partial \theta_a} \frac{\partial E_k(\textbf{R})}{\partial R_j} \Big] + \Big\langle \psi_{k}(\boldsymbol\theta^{k}(\textbf{R})) \Big| \frac{\partial^2 H(\textbf{R})}{\partial R_i \partial R_j} \Big| \psi_{k}(\boldsymbol\theta^{k}(\textbf{R})) \Big\rangle
$$

where $|\psi_k(\theta)\rangle = U_k(\theta) |0\rangle$ is a variational ansatz which is used to prepare the $k$-th excited state, and $\boldsymbol{\theta}^k(\textbf{R})$ are the variational parameters of $U_k(\theta)$ which prepare the $k$-th energy eigenstate at coordinates $\textbf{R}$. In addition, by Feynman-Hellmann theorem, we have

$$
\frac{\partial E_k(\textbf{R})}{\partial R_j} = \Big\langle \psi_k( \boldsymbol\theta^{k}(\textbf{R})) \Big| \frac{\partial H(\textbf{R})}{\partial R_j} \Big| \psi_k(\boldsymbol\theta^{k}(\textbf{R})) \Big\rangle
$$

Finally, we can determine the derivatives of the optimal parameters $\boldsymbol{\theta}^{k}$ with the response equation:

$$
\displaystyle\sum_{a} \frac{\partial}{\partial \theta_b} \frac{\partial E_k(\textbf{R})}{\partial \theta_a} \frac{\partial \boldsymbol{\theta}^{k}_a(\textbf{R})}{\partial R_i} = -\frac{\partial}{\partial \theta_b} \frac{\partial E_k(\textbf{R})}{\partial R_i}
$$

We begin by computing the first derivatives of the molecular Hamiltonian:

In [8]:
# Generates first and second derivatives of the Hamiltonian
H1 = hf.generate_d_hamiltonian(mol, wires, bar=True)(geometry.flatten())

  0%|          | 0/9 [00:00<?, ?it/s]

/Users/jackceroni/Library/Python/3.9/lib/python/site-packages/autohf-0.0.1-py3.9.egg/autohf/hartreefock.py:209: RuntimeWarning: divide by zero encountered in true_divide


Next, we compute the matrix of second derivatives (with respect to variational parameters) of the energy function:

In [9]:
# Non-sparse Hamiltonian for computing second derivatives
h_new = hf.H(mol, wires)(geometry.flatten())

# Energy function
@qml.qnode(dev)
def energy_fn(params):
    circuit(params)
    return qml.expval(h_new)

# Parameter Hessian
hessian = tg.compute_parameter_hessian(energy_fn, np.array(original_params), bar=True)

  0%|          | 0/18 [00:00<?, ?it/s]

Finally, we can compute the second derivative of the Hamiltonian, and the desired energy derivative. In particular, we will compute $\frac{\partial E}{\partial R_1^2}$, where $R_1$ is the $y$-coordinate of the Be atom:

In [10]:
# Hamiltonian second derivative
H2 = hf.generate_dd_hamiltonian(mol, wires, bar=True)(geometry.flatten())

  0%|          | 0/9 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in true_divide
  return f_raw(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/autograd/numpy/numpy_jvps.py:46: RuntimeWarning: invalid value encountered in true_divide
  lambda g, ans, x, y : - g * x / y**2)


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [11]:
# Energy derivative
non_tailgated_hessian = tg.hessian(H1, H2, circuit, dev, original_params, hessian, bar=True, diff_method="best")

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

We can then compute the normal mode frequencies by simply finding the eigenvectors and eigenvalues of the Hessian:

In [19]:
m = [15.999, 1.00784, 1.00784] # Masses of atoms
vecs, vals = tg.vec_vals(non_tailgated_hessian, m) # Normal modes and frequencies

print("Non-tailgated normal mode frequencies = {}".format([float(x) for x in vals]))

Non-tailgated normal mode frequencies = [nan, 4890.568242933075, 2301.175987139119, 978.101347057233, 978.1012531465711, 47.78423310331804, 47.78420182693933, 0.000411140342652434, 1.3819259784933116e-06]


/var/folders/yd/_p7ptfl11dqffr29bbsh4wyw0000gn/T/ipykernel_8612/1753049031.py:33: RuntimeWarning: invalid value encountered in sqrt
  f = np.sqrt(vals) * conv


### Step 3. Performing tailgated ADAPT-VQE

We will now demonstrate that tailgating yields the correct frequencies. We perform the tailgating procedure, as is described in the paper:

In [22]:
# Defines a flat version of the gate pool of single and double excitations
flat_gate_pool = gate_pool[0]
flat_gate_pool.extend(gate_pool[1])

In [47]:
# Performs the tailgating procedure
hamiltonians = [H1[0], H1[1], H1[2], H1[3], H1[4], H1[5]]

# A method for tailgating a circuit
def tailgate(dH, dev, circuit, operator_pool, params, tol=1e-6):
    """Performs the tail-gating procedure"""
    gates = []
    bar = tqdm(operator_pool)
    n = []

    for op in bar:
        @qml.qnode(dev)
        def cost_fn(param):
            circuit(params)
            op(param[0])
            return qml.expval(dH)
        gr = qml.grad(cost_fn)(np.array([0.0]))
        if abs(gr) > tol:
            gates.append(op)
        else:
            n.append(op)
    return n, gates

# Gates added to the circuit through tailgating
added_gates = []
for dh in hamiltonians:
    flat_gate_pool, gates = tailgate(dh, dev, circuit, flat_gate_pool, np.array(original_params))
    added_gates.extend(gates)

added_gates = set(added_gates)

  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Now, we can construct a new, tailgated circuit:

In [49]:
# Creates the new gate pool
new_seq = original_seq + list(added_gates)
new_params = np.array(list(original_params) + [0.0 for _ in range(len(added_gates))])

# Circuit yielded from tailgating
def new_circuit(params):
    qml.BasisState(hf_state, wires=wires)
    for p, gate in zip(params, new_seq):
        gate(p)

Once again, we compute the molecular Hessian, this time using the tailgated circuit:

In [50]:
# Energy function
@qml.qnode(dev)
def new_energy_fn(params):
    new_circuit(params)
    return qml.expval(h_new)

# Parameter Hessian
new_hessian = tg.compute_parameter_hessian(new_energy_fn, np.array(new_params), bar=True)

  0%|          | 0/76 [00:00<?, ?it/s]

In [51]:
# Energy derivative
tailgated_hessian = tg.hessian(H1, H2, new_circuit, dev, new_params, new_hessian, bar=True, diff_method="best")

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

which gives us the following frequencies:

In [52]:
new_vecs, new_vals = vec_vals(tailgated_hessian, m)

print("Tailgated normal mode frequencies = {}".format([float(x) for x in new_vals]))

Tailgated normal mode frequencies = [nan, 2457.881073140677, 2301.1823938585608, 745.7149476037524, 745.6925286829708, 47.78423310331241, 47.78420182693795, 0.0004110108597292169, 5.418640536228329e-06]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in sqrt
  return f_raw(*args, **kwargs)


These frequencies, as can be checked, are much closer to the values yielded by GAMESS than the frequencies given by the non-tailgated circuit.